In [16]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import datasets,transforms

In [5]:
# Lenet5 model
class CNN_Model(nn.Module):
    def __init__(self):
        super(CNN_Model,self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5,padding=2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.conv3 = nn.Conv2d(16,120,kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(120,84)
        self.fc2 = nn.Linear(84,10)
        self.logsoftmax = nn.LogSoftmax()
        
    def forward(self,x):
        in_size = x.size(0)
        out = self.relu(self.mp(self.conv1(x)))
        out = self.relu(self.mp(self.conv2(out)))
        out = self.relu(self.conv3(out))
        out = out.view(in_size, -1)
        out = self.relu(self.fc1(out))
        out = self.fc2(out)
        return self.logsoftmax(out)

In [18]:
device = torch.device("cuda")

In [20]:
# Load model
model = CNN_Model()
model.load_state_dict(torch.load("./model/MNIST/Lenet5_model.pt"))
model.cuda()
model.eval()

CNN_Model(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
  (mp): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu): ReLU()
  (fc1): Linear(in_features=120, out_features=84, bias=True)
  (fc2): Linear(in_features=84, out_features=10, bias=True)
  (logsoftmax): LogSoftmax(dim=None)
)

In [9]:
print(model.state_dict())

OrderedDict([('conv1.weight', tensor([[[[ 0.0695,  0.1512, -0.0573, -0.3021, -0.0715],
          [ 0.1828, -0.0799,  0.0251, -0.2022, -0.0347],
          [-0.1576,  0.1610,  0.4129,  0.3227,  0.2098],
          [ 0.0430,  0.1009,  0.1638, -0.0104, -0.0247],
          [-0.3150, -0.1830, -0.2694, -0.0310, -0.0054]]],


        [[[-0.2107, -0.4379, -0.1823,  0.1826,  0.2716],
          [-0.4535, -0.3375,  0.0900,  0.3838,  0.4194],
          [-0.4674, -0.1945,  0.1568,  0.3870,  0.2816],
          [-0.1398, -0.0228,  0.3857,  0.3888, -0.0967],
          [ 0.0284,  0.2243,  0.1359,  0.2529,  0.1260]]],


        [[[-0.1100, -0.0511,  0.1732, -0.0481, -0.3081],
          [ 0.0027, -0.0874,  0.0449, -0.1961, -0.4010],
          [ 0.2055, -0.0137,  0.1291, -0.0656, -0.0158],
          [ 0.0088,  0.3631,  0.1364,  0.0683, -0.1210],
          [-0.1665,  0.3177,  0.3586,  0.1550, -0.0549]]],


        [[[ 0.3058,  0.2059,  0.2173, -0.2197,  0.0408],
          [ 0.3883,  0.0301, -0.0161, -0.1616,

In [13]:
test_dataset = datasets.MNIST('data/',download=False,train=False,
                              transform=transforms.Compose([
                                  transforms.ToTensor(),
                                  transforms.Normalize((0.5,), (0.5,)),
                              ]))

In [14]:
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=64,shuffle=True)

In [22]:
correct = 0
for image, label in test_loader:
    image, label = Variable(image), Variable(label)
    image = image.to(device)
    label = label.to(device)
    output = model(image)
    # get the index of the max log-probability
    # keepdim：讓pred和output維度相同
    #[0]代表最大值,[1]代表最大值的位置
    pred = output.data.max(1, keepdim=True)[1]
    #torch.eq：在相同位置返回True，否則返回False 
    #view_as：使label和pred同個shape
    correct += pred.eq(label.data.view_as(pred)).cpu().sum()
print('{:.3f}%\n'.format(
    100. * correct / len(test_loader.dataset)))

C:\Users\x9172\anaconda3\envs\PT\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


98.810%

